In [2]:

%load_ext autoreload
%autoreload 2
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np
import torch

from transformers import pipeline

In [3]:
from transformers import GPT2Config, GPT2Model,PreTrainedModel,PretrainedConfig
from MoEWrapper import MoEWrapper
cfg=PretrainedConfig()
cfg=GPT2Config()

moe=MoEWrapper(cfg)

model_name_list =[ 
    "llm-jp/llm-jp-13b-instruct-full-jaster-v1.0",
    "llm-jp/llm-jp-13b-instruct-full-dolly_en-dolly_ja-ichikara_003_001-oasst_en-oasst_ja-v1.1",
                  ]

tokenizer = AutoTokenizer.from_pretrained(model_name_list[0])
for model_name in model_name_list:
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto",
        torch_dtype=torch.float16
    )
    moe.append_model(model)

In [19]:
max_new_tokens=10
repetition_penalty=2.0
pipe=pipeline("text-generation",model=moe,
              tokenizer=tokenizer,
     max_new_tokens=max_new_tokens,
                      repetition_penalty=repetition_penalty,
 pad_token_id=50256,
              )



The model 'MoEWrapper' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM

In [31]:
inputs=[
    "hello, I'm John",
    "global warming ",
    "英語: He is a good man. 日本語: ",
    "今日は晴れてるから",
    "ブログを書きました",
    "地球温暖化を防ぐに",
    "ガリレオによるピサの斜塔実験とは",
    "スーパーコンピューターの富岳は",
]
moe.set_flexible_mode()
for text in inputs:
    print("------")
    print("input: ", text)
    print(pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model 2 will be used
ppl array:  [338.5   186.125 172.125]


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


hello, I'm John."  "I Love You Like That (
------
input:  global warming 
model 2 will be used
ppl array:  [158.625 162.375 150.75 ]
global warming  The climate of the United States is characterized by
------
input:  英語: He is a good man. 日本語: 
model 2 will be used
ppl array:  [34.71875 36.3125  33.6875 ]
英語: He is a good man. 日本語: 私は、私の夫は私を愛している。
------
input:  今日は晴れてるから
model 1 will be used
ppl array:  [227.25 191.25 326.75]
今日は晴れてるから、ちょっとだけ散歩してきます。 20
------
input:  ブログを書きました
model 1 will be used
ppl array:  [1746.   538.5  960. ]
ブログを書きました。 2014年5月3
------
input:  地球温暖化を防ぐに
model 0 will be used
ppl array:  [152.5  164.25 304.5 ]
地球温暖化を防ぐにはどうしたらいいか? 2017
------
input:  ガリレオによるピサの斜塔実験とは
model 2 will be used
ppl array:  [290.5   146.125 141.125]
ガリレオによるピサの斜塔実験とは、1960年代後半にイタリアで
------
input:  スーパーコンピューターの富岳は
model 2 will be used
ppl array:  [1307. 1571.  601.]
スーパーコンピューターの富岳は、2017年3月で稼働


In [32]:
for text in inputs:
    print("------")
    print("input: ", text)
    for model_id in range(len(model_name_list)):
        moe.fix_model(model_id)
        print(f"model id {model_id}: ",pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model id 0:  hello, I'm John. 2016年3月 
model id 1:  hello, I'm John. I am not sure if you have any
model id 2:  hello, I'm John."  "I Love You Like That (
------
input:  global warming 
model id 0:  global warming 2016年3月 【お知らせ
model id 1:  global warming 2018-5 Posted by
model id 2:  global warming  The climate of the United States is characterized by
------
input:  英語: He is a good man. 日本語: 
model id 0:  英語: He is a good man. 日本語: 「彼は私のことを愛している」 20
model id 1:  英語: He is a good man. 日本語: Thank you for your kindness and help in finding
model id 2:  英語: He is a good man. 日本語: 私は、私の夫は私を愛している。
------
input:  今日は晴れてるから
model id 0:  今日は晴れてるから、ちょっとだけ雨降ってた。 でも
model id 1:  今日は晴れてるから、ちょっとだけ散歩してきます。 20
model id 2:  今日は晴れてるから、また行こうかな。 2017
------
input:  ブログを書きました
model id 0:  ブログを書きました。 2017年3月9
model id 1:  ブログを書きました。 2014年5月3
model id 2:  ブログを書きました。 2017年6月3
------
input:  地球温暖化を防ぐに
model id 0:  地球温暖化を防ぐにはどうしたらいいか? 2017
model id 1:  地球温暖化を防ぐにはどうすればいいか? 2018
mo